Get pokemon data from pokeAPI

In [1]:
import requests
import json
import os

# 定义保存图片的目录
save_directory = "data/pokemon_images"
os.makedirs(save_directory, exist_ok=True)

# 获取所有宝可梦的基本信息
url = "https://pokeapi.co/api/v2/pokemon?limit=10000"
response = requests.get(url)
data = response.json()

# 创建一个字典来存储宝可梦的名称和属性
pokemon_info = {}

# 遍历每个宝可梦
for pokemon in data['results']:
    pokemon_name = pokemon['name']
    pokemon_url = pokemon['url']
    response = requests.get(pokemon_url)
    pokemon_data = response.json()

    # 获取宝可梦的属性
    types = pokemon_data['types']
    type_names = [type_info['type']['name'] for type_info in types]

    # 获取宝可梦的图片链接
    pic1_url = pokemon_data['sprites']['other']['official-artwork']['front_default']
    pic2_url = pokemon_data['sprites']['other']['home']['front_default']
    pic3_url = pokemon_data['sprites']['front_default']

    # 下载并保存图片
    for i, pic_url in enumerate([pic1_url, pic2_url, pic3_url], start=1):
        if pic_url:
            img_response = requests.get(pic_url)
            if img_response.status_code == 200:
                img_name = f"{pokemon_name}_pic{i}.png"
                img_path = os.path.join(save_directory, img_name)
                with open(img_path, 'wb') as file:
                    file.write(img_response.content)

    # 将宝可梦的名称和属性存储到字典中
    pokemon_info[pokemon_name] = {
        "types": type_names,
        "images": [os.path.join(save_directory, f"{pokemon_name}_pic{i}.png") for i in range(1, 4) if eval(f"pic{i}_url")]
    }

# 将字典转换为 JSON 格式并保存到文件中
with open('data/pokemon_info.json', 'w') as json_file:
    json.dump(pokemon_info, json_file, indent=4)

print(f"所有宝可梦的名称和属性已保存到 pokemon_info.json 文件中，图片保存在 {save_directory} 目录中。")

所有宝可梦的名称和属性已保存到 pokemon_info.json 文件中，图片保存在 data/pokemon_images 目录中。


In [16]:
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image

# 定义自定义数据集类
class PokemonDataset(Dataset):
    def __init__(self, data_dict, transform=None):
        self.data_dict = data_dict
        self.transform = transform
        self.image_files = []
        self.labels = []
        
        # 遍历数据字典，获取所有图像文件路径和对应的标签（属性）
        for pokemon_name, info in data_dict.items():
            for img_path in info['images']:
                self.image_files.append(img_path)
                self.labels.append(info['types'])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert("RGB")
        
        # 获取宝可梦的标签（属性）
        labels = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        # 将字符串标签转换为整数标签
        label_map = {'grass': 0, 'poison': 1, 'fire': 2, 'water': 3, 'electric': 4, 'ice': 5, 'fighting': 6,
                     'ground': 7, 'flying': 8, 'psychic': 9, 'bug': 10, 'rock': 11, 'ghost': 12, 'dark': 13,
                     'dragon': 14, 'steel': 15, 'fairy': 16, 'normal': 17}
        labels = [label_map[label] for label in labels]
        
        # 将标签转换为多标签二进制格式
        multi_label = torch.zeros(len(label_map), dtype=torch.float32)
        for label in labels:
            multi_label[label] = 1.0
        
        return image, multi_label

# 读取数据字典
with open('data/pokemon_info.json', 'r') as f:
    data_dict = json.load(f)

# 定义训练数据的转换
data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

# 创建数据集
pokemon_dataset = PokemonDataset(data_dict=data_dict, transform=data_transforms)

# 将数据集分割为训练集、验证集和测试集（80%、10%、10%）
train_size = int(0.8 * len(pokemon_dataset))
val_size = int(0.1 * len(pokemon_dataset))
test_size = len(pokemon_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(pokemon_dataset, [train_size, val_size, test_size])

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)


In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 加载预训练的ResNet18模型
model = models.resnet18(pretrained=True)

# 修改最后一层以匹配属性的数量（假设有18种可能的属性）
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 18)

# 将模型移动到GPU
model = model.to(device)

# 定义损失函数和优化器
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 训练循环
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        # 将输入和标签移动到GPU
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

    # 验证循环
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            # 将输入和标签移动到GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    
    print(f"Validation Loss: {val_loss/len(val_loader)}")

print("Training complete.")

Using device: cuda:0


UnidentifiedImageError: cannot identify image file 'E:\\projects\\Pokemon_type\\data\\pokemon_images\\meowstic-male_pic3.png'